In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
{'lr': 0.0012098123619624396,
 'layers': 2,
 'step_size': 21,
 'gamma': 0.5302067528042456,
 'bptt': 12,
 'dropout': 0.35583243487203325}

{'lr': 0.0012098123619624396,
 'layers': 2,
 'step_size': 21,
 'gamma': 0.5302067528042456,
 'bptt': 12,
 'dropout': 0.35583243487203325}

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/lga.csv')

In [5]:
dataset.shape

(8757, 1045)

In [6]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
1,2018-01-01,4,1,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2018-01-01,5,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

774

In [8]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 271)

In [9]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

257

In [10]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [11]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 271)
(2190, 271)


In [12]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 257])
torch.Size([2190, 13])
torch.Size([2190, 257])


In [13]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
#         train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq_x, train_seq_y ,train_label))
    return inout_seq

In [14]:
bptt = 12

In [15]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [16]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape, train_inout_seq[0][2].shape

(torch.Size([12, 13]), torch.Size([12, 257]), torch.Size([12, 257]))

In [17]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [18]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=100, num_layers=1, dropout=0):
        super().__init__()
                
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.hidden_cell = (torch.zeros(num_layers,1,self.hidden_size).to(device),
                    torch.zeros(num_layers,1,self.hidden_size).to(device))
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)


    def forward(self, x):
        self.hidden_cell = (torch.zeros(self.num_layers,1,self.hidden_size).to(device),
                    torch.zeros(self.num_layers,1,self.hidden_size).to(device))
           
        lstm_out, self.hidden_cell = self.lstm(x.view(len(x) ,1, -1), self.hidden_cell)
        
        return lstm_out, self.hidden_cell

In [19]:
class GraphPrediction(nn.Module):
    def __init__(self, feat_size=1, hidden_layer_size=100, network_size=1, layers=1, communities=10, ensembles=1, dropout=0):
        super().__init__()
        
        # aggregation
        self.attachment_matrix = torch.nn.Parameter(torch.randn(network_size,communities))
        self.attachment_matrix.requires_grad = True
        
        lstm_input = communities + feat_size
        
        self.ensembles = ensembles
        self.lstms = nn.ModuleList()
        self.linears = nn.ModuleList()
        for i in range(ensembles):
             self.lstms.append(LSTM(input_size=lstm_input, hidden_size=hidden_layer_size, num_layers=layers))
             self.linears.append(nn.Linear(hidden_layer_size, network_size))
            

    def forward(self, input_seq, feat):
        
        w = F.softmax(self.attachment_matrix, dim=1)
        x = torch.matmul(input_seq, self.attachment_matrix)
        x = torch.cat((x,feat),axis=1)
        x = x.view(len(input_seq) ,1, -1)
        
        predictions = []
        for i in range(self.ensembles):
            if torch.randn(1) < 0.7 or i==0 or not self.training:
                lstm_out, self.hidden_cell = self.lstms[i](x)
                y = self.linears[i](lstm_out.view(len(input_seq), -1))
                predictions.append(y)
        
        predictions = torch.stack(predictions)
#         print(predictions.shape)
        return predictions

In [20]:
def evaluate(model):
    model.eval()
    prediction = []
    with torch.no_grad():
        for feat,seq, labels in test_inout_seq:
#             model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
#                             torch.zeros(layers, 1, model.hidden_layer_size))
            y = model(seq,feat).mean(dim=0)[-1]
    
            prediction.append(y)

    y_test_ = torch.stack([labels[-1] for feat,seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
    y_pred_ = torch.stack(prediction).detach().cpu().numpy()

    r2 = r2_score(y_test_, y_pred_, multioutput='variance_weighted')
    rmse = mean_squared_error(y_test_, y_pred_)
    mae = mean_absolute_error(y_test_, y_pred_)
#     print("r2: ",r2)
    return (r2, rmse, mae)

In [21]:
layers = 2
communities = 24
network_size = len(targetColumns)
feat_size = len(features_cols)
ensembles=10
dropout = 0.35583243487203325

model = GraphPrediction(feat_size = feat_size, hidden_layer_size=100,
             network_size=network_size, layers=layers,
            communities=communities, ensembles=ensembles, dropout=dropout).to(device)



In [22]:
loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.0012098123619624396)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=21, gamma=0.53)

In [ ]:
epochs = 200


for i in range(epochs):
    model.train()
    for feat,seq, labels in train_inout_seq:
        optimizer.zero_grad()
#         model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
#                         torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq, feat)
        labels = labels.repeat(y_pred.shape[0],1,1)
        
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        
    scheduler.step()
#     if i%1 == 1:
    r2, rmse, mae = evaluate(model)
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')
    torch.save(model.state_dict(), 'lga.pt')
    
print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   0 loss: 1.38645327 r2: 0.687 rmse: 7.290 mae: 1.122
epoch:   1 loss: 1.33436620 r2: 0.710 rmse: 6.769 mae: 1.080
epoch:   2 loss: 1.32500970 r2: 0.717 rmse: 6.597 mae: 1.075
epoch:   3 loss: 1.27309620 r2: 0.731 rmse: 6.281 mae: 1.056
epoch:   4 loss: 1.28326321 r2: 0.734 rmse: 6.202 mae: 1.047
epoch:   5 loss: 1.25893557 r2: 0.737 rmse: 6.124 mae: 1.045
epoch:   6 loss: 1.23533165 r2: 0.742 rmse: 6.008 mae: 1.034
epoch:   7 loss: 1.24017417 r2: 0.739 rmse: 6.075 mae: 1.033
epoch:   8 loss: 1.25892913 r2: 0.744 rmse: 5.957 mae: 1.033
epoch:   9 loss: 1.25921524 r2: 0.746 rmse: 5.929 mae: 1.031
epoch:  10 loss: 1.20652556 r2: 0.752 rmse: 5.781 mae: 1.022
epoch:  11 loss: 1.24395871 r2: 0.748 rmse: 5.868 mae: 1.024
epoch:  12 loss: 1.22825789 r2: 0.751 rmse: 5.799 mae: 1.024
epoch:  13 loss: 1.22031760 r2: 0.749 rmse: 5.854 mae: 1.022
epoch:  14 loss: 1.19806409 r2: 0.755 rmse: 5.719 mae: 1.018
epoch:  15 loss: 1.22132039 r2: 0.754 rmse: 5.743 mae: 1.019
epoch:  16 loss: 1.22987

epoch: 135 loss: 1.05706239 r2: 0.769 rmse: 5.393 mae: 0.993


In [ ]:
# best value 0.771

In [ ]:
evaluate(model)

In [ ]:
attachment = torch.argmax(F.softmax(model.attachment_matrix, dim=1), dim=1).detach().cpu().numpy().astype(str)
community_assignment = dict(zip(targetColumns, attachment))
community_assignment

In [ ]:
import json

with open('lga_attachment.json', 'w') as fp:
    json.dump(community_assignment, fp)

In [ ]:
len(set(attachment))

In [ ]:
# 2 linear
# 0.47

In [ ]:
# 1 linear
# 0.51

In [ ]:
# 1 linear + RELU
# 0.50

In [ ]:
# 1 linear bptt = 24
# 0.52

In [ ]:
#ensemble
#0.53228

In [ ]:
#  TODO
# other hubs
# ensemble model